In [5]:
#os.listDir : liste tous les éléments à partir du chemin donnée
#os.path.join : concatenne les chemins


import os
import csv
import imageio
import numpy
import matplotlib.pyplot as plt
import torchvision as torch
import numpy as np
from PIL import Image
from posix import write
from tqdm import tqdm
from torch.utils.data import Dataset
import torchvision
from torchvision.io import read_image, ImageReadMode
import pandas as pd

In [14]:
# import archive.zip dans l'ordi
! 7z x /content/archive.zip


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/                   1 file, 728990397 bytes (696 MiB)

Extracting archive: /content/archive.zip
--
Path = /content/archive.zip
Type = zip
Physical Size = 728990397

  0%      6% 48 - Dataset/0/Colour/6.jpg                                 10% 79 - Dataset/11/Colour/33.jpg                                   13% 110 - Dataset/15/Colour/28.jpg                                    16% 132 - Dataset/16/Colour/15.jpg                                   

In [15]:
def initialiseDataSet() :
  CHEMIN = "/content/Dataset/"
  try :
    fichierCSV = open("/content/fichier.csv",'w+',newline='',encoding='utf-8')
  except :
    print("erreur création fichier fichier.csv")
  csvVersion = csv.writer(fichierCSV)
  csvVersion.writerow(["image_normal","image_BW"])
  for volume in os.listdir(CHEMIN) :
    volumePath = os.path.join(CHEMIN,volume)
    cheminVolumeNoirBlanc = os.path.join(volumePath,"BlackAndWhiteImage")
    try :
      os.mkdir(cheminVolumeNoirBlanc)
    except  :
      print("erreur création dossier")
    imagesPath = os.listdir(os.path.join(volumePath,"Colour"))
    for imagePath in imagesPath :
      cheminImageGris = "BW_" + imagePath
      colorVolumePath = os.path.join(volumePath,"Colour",imagePath)
      blackWhiteVolumePath = os.path.join(cheminVolumeNoirBlanc,cheminImageGris)
      csvVersion.writerow([colorVolumePath,blackWhiteVolumePath])
      imageNormal = Image.open(colorVolumePath)
      imageBlackWhite = imageNormal.convert('L')
      imageBlackWhite.save(blackWhiteVolumePath)



In [44]:
class CustomDataSet(Dataset):
    def __init__(self, csv_file, size):
        """
        Classe CustomDataSet pour charger des images et les transformer.

        Args:
            csv_file (str): Chemin du fichier CSV contenant les chemins des images.
            size (tuple): Taille (hauteur, largeur) pour redimensionner les images.
        """
        super().__init__()
        self.csv_file = csv_file
        self.size = size

        # Lire tous les chemins du CSV au moment de l'initialisation
        with open(csv_file, 'r') as f:
            reader = csv.reader(f)
            self.image_paths = list(reader)
            self.image_paths.pop(0)

    def __len__(self):

        return len(self.image_paths)

    def __getitem__(self, index: int) :
        """
        Charge une paire d'images (couleur et noir et blanc), applique les transformations nécessaires.

        Args:
            index (int): Index de l'image à charger.

        Returns:
            tuple: Tensor de l'image couleur, Tensor de l'image noir et blanc.
        """
        # Récupérer les chemins des images pour l'index donné
        chemin_image_couleur, chemin_image_bw = self.image_paths[index]

        # Lire les images
        image_couleur = read_image(chemin_image_couleur, ImageReadMode.RGB).float()
        image_bw = read_image(chemin_image_bw, ImageReadMode.RGB).float()

        # Appliquer les transformations
        resizer = torchvision.transforms.Resize(self.size)
        image_couleur = resizer.forward(image_couleur)
        image_bw = resizer.forward(image_bw)

        # Normaliser les valeurs entre -1 et 1
        image_couleur = (image_couleur / 127.5) - 1.0
        image_bw = (image_bw / 127.5) - 1.0

        return image_couleur, image_bw
    def __repr__(self) -> str:
        return f"""
          Il y a {self.__len__()} images.
          Les images sont de taille : {self.size}
        """

In [27]:
def estInitialise() :
  try :
    os.mkdir("/content/Dataset")
  except :
    try:
      os.mkdir("/content/Dataset/0/BlackAndWhiteImage")
    except :
      return len(os.listdir("/content/Dataset/0/BlackAndWhiteImage")) > 0

In [46]:
if __name__ == "__main__" :
  if not estInitialise() :
    initialiseDataSet()
  dataset = CustomDataSet("/content/fichier.csv",(256,256))
  print(len(dataset))
  print(dataset)

835

          Il y a 835 images.
          Les images sont de taille : (256, 256)
        
